In [6]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

In [7]:
def read_csvs(path):
    data = pd.DataFrame()
    for root, dir, files in os.walk(path):
        for file in files:
            if file.endswith(".csv"):
                data = pd.concat([data, pd.read_csv(os.path.join(root, file))])
    return data

df = read_csvs("out/csv/squat")

In [8]:
x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [9]:
x.shape

(4172, 12)

In [10]:
# x = x.reshape(len(x), 1, x.shape[1])

In [11]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.8, test_size=0.2, random_state=1)

In [13]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Reshape
from keras.optimizers import Adam

model = Sequential([
    Reshape((1, x.shape[1]), input_shape=(x.shape[1],)), # 12 is the number of features
    # LSTM(128, activation='relu', return_sequences=True),
    # LSTM(32, activation='relu'),
    GRU(128, activation='relu', return_sequences=True),
    GRU(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

opt_adam = Adam(learning_rate=1e-3)

model.compile(optimizer=opt_adam, loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(x_train, y_train, epochs=200, validation_data=(x_test, y_test))

Epoch 1/200
105/105 [==============================] - 5s 10ms/step - loss: 0.6825 - accuracy: 0.5433 - val_loss: 0.6614 - val_accuracy: 0.5677
Epoch 2/200
105/105 [==============================] - 0s 4ms/step - loss: 0.6404 - accuracy: 0.6221 - val_loss: 0.6619 - val_accuracy: 0.6443
Epoch 3/200
105/105 [==============================] - 0s 5ms/step - loss: 0.6325 - accuracy: 0.6338 - val_loss: 0.6395 - val_accuracy: 0.6455
Epoch 4/200
105/105 [==============================] - 0s 4ms/step - loss: 0.6132 - accuracy: 0.6449 - val_loss: 0.6394 - val_accuracy: 0.5713
Epoch 5/200
105/105 [==============================] - 0s 4ms/step - loss: 0.6201 - accuracy: 0.6428 - val_loss: 0.6100 - val_accuracy: 0.6635
Epoch 6/200
105/105 [==============================] - 0s 4ms/step - loss: 0.6055 - accuracy: 0.6530 - val_loss: 0.6117 - val_accuracy: 0.6263
Epoch 7/200
105/105 [==============================] - 0s 4ms/step - loss: 0.6067 - accuracy: 0.6506 - val_loss: 0.6048 - val_accuracy: 0.665

In [15]:
model.save('out/models/model3.h5')

In [47]:
from keras.models import save_model

save_model(model, 'out/models/android.h5')

In [48]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tfmodel = converter.convert()
open('out/models/android.tflite', 'wb').write(tfmodel)

INFO:tensorflow:Assets written to: C:\Users\SIDDHA~1\AppData\Local\Temp\tmpnysh_pl0\assets


INFO:tensorflow:Assets written to: C:\Users\SIDDHA~1\AppData\Local\Temp\tmpnysh_pl0\assets


390348